## CIFAR 10

In [1]:
import fastai.models as mod

In [2]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [3]:
from fastai.conv_learner import *
PATH = "../fp16/data/cifar10/"
os.makedirs(PATH,exist_ok=True)

### Load Data

In [4]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
stats = (np.array([ 0.4914 ,  0.48216,  0.44653]), np.array([ 0.24703,  0.24349,  0.26159]))

In [5]:
def get_data(sz,bs):
    tfms = tfms_from_stats(stats, sz, aug_tfms=[RandomFlip()], pad=sz//8)
    return ImageClassifierData.from_paths(PATH, val_name='test', tfms=tfms, bs=bs)

In [6]:
bs=128

## Initial model

In [7]:
%pdb on

Automatic pdb calling has been turned ON


In [24]:
from fastai.models.cifar10.resnext import resnext29_8_64

m = resnext29_8_64()
bm = BasicModel(m.cuda(), name='cifar10_rn29_8_64')
data = get_data(32,bs)

In [25]:
learn = ConvLearner(data, bm)
learn.unfreeze()
lr=1e-2; wd=5e-4

In [26]:
learn.half()

In [20]:
# learn.model.half()

In [21]:
# learn.load('fp32_model')

In [27]:
torch.backends.cudnn.benchmark=True

In [23]:
%time learn.fit(lr, 1, cycle_len=.25, fp16=False)

98it [00:46,  2.10it/s, loss=2.4]                         epoch      trn_loss   val_loss   accuracy   
    0      2.39926    2.112249   0.346816  

CPU times: user 48.4 s, sys: 20.4 s, total: 1min 8s
Wall time: 58.7 s


[2.112248773816266, 0.346815664556962]

In [28]:
%time learn.fit(lr, 1, cycle_len=.25)

98it [01:15,  1.30it/s, loss=2.25]                        epoch      trn_loss   val_loss   accuracy   
    0      2.24627    1.880303   0.323774  

CPU times: user 1min 25s, sys: 29.6 s, total: 1min 55s
Wall time: 1min 43s


[1.8803031052215189, 0.32377373417721517]

In [ ]:
# %time learn.fit(lr, 1, cycle_len=.25, loss_scale=128, fp16=True)

In [ ]:
%time learn.fit(lr, 1, cycle_len=.25, loss_scale=128, fp16=False)

In [ ]:
learn.model

In [ ]:
learn.save('fp16_model')

## Testing 32

In [ ]:
from fastai.models.cifar10.resnext import resnext29_8_64

m = resnext29_8_64()
bm = BasicModel(m.cuda(), name='cifar10_rn29_8_64')
data = get_data(32,bs)

In [ ]:
learn = ConvLearner(data, bm)
learn.unfreeze()
lr=1e-2; wd=5e-4

In [ ]:
learn.load('fp16_model')

In [ ]:
%time learn.fit(lr, 1, cycle_len=.25)

In [ ]:
learn.save('fp32_model')